In [17]:
test_semantic_model_url = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/semantic_model.txt' 
questions_url           = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/questions.json'
SF                      = 0.1
max_attempts            = 2
TIMEOUT_SECONDS         = 180  
output_dir              = "/tmp/llm"
model1                  = "o3-mini"
model2                  = "qwen3:30b-a3b"

In [18]:
import requests
import json
import duckdb
import pathlib
import time
import re
from   openai import AzureOpenAI, OpenAIError
import datetime
import os
import threading
import pandas as pd


# Generate Data

In [19]:
if SF <1 :
 schema = f"DS{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
os.makedirs(output_dir, exist_ok=True)
db_path = output_dir +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach '{db_path}' as ds(read_only) ; use ds """)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Run Tests

In [20]:
def get_ai_response(user_message, LLM, provider, api_key=None, azure_endpoint=None, azure_api_version=None):
    system_prompt = ""
    try:
        github_response = requests.get(test_semantic_model_url)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        print(f"Error fetching system prompt from GitHub ({test_semantic_model_url}): {e}")
        return f"Error fetching system prompt: {e}"

    generated_text = None
    data = None

    if provider == "ollama":
        url = 'http://127.0.0.1:11434/api/chat'
        headers = {'Content-Type': 'application/json'}
        payload = {
            'model': LLM,
            'messages': [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_message}
            ],
            'stream': False
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            generated_text = data.get('message', {}).get('content', '')
            if generated_text:
                generated_text = generated_text.replace("<think>", "").replace("</think>", "")
        except requests.RequestException as e:
            return f"Error with Ollama API request: {e}"
        except (KeyError, IndexError, AttributeError) as e:
            return f"Unexpected response format from Ollama: {e}\nFull response data: {data}"

    elif provider == "azure_foundry":
        if not azure_endpoint or not azure_api_version or not api_key:
            return "Azure provider requires 'azure_endpoint', 'azure_api_version', and 'api_key' parameters."

        try:
            client = AzureOpenAI(
                api_version=azure_api_version,
                azure_endpoint=azure_endpoint,
                api_key=api_key,
            )

            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                model=LLM,
            )

            generated_text = response.choices[0].message.content

        except Exception as e:
            return f"An error occurred with Azure OpenAI: {e}"



    else:
        return f"Unsupported provider for this test setup: {provider}. Supported providers are 'ollama', 'azure_foundry', and 'gemini'."

    if not isinstance(generated_text, str):
         print(f"Generated text is not a string (type: {type(generated_text)}). Cannot clean.")
         if generated_text is None:
              return "Could not retrieve generated text from provider response."
         else:
              return f"Received unexpected output type from {provider}: {type(generated_text)}. Value: {generated_text}"


    cleaned_text = re.sub(r'```(sql|duckdb)?\s*([\s\S]*?)\s*```', r'\2', generated_text, flags=re.IGNORECASE).strip()

    if "```" in cleaned_text or "SELECT " in cleaned_text.upper() or "WITH " in cleaned_text.upper():
         cleaned_text = cleaned_text.replace("```sql", "").replace("```duckdb", "").replace("```", "").strip()


    return cleaned_text

In [21]:
def execute_sql_with_retry(query, test_model, provider, api_key, azure_endpoint, azure_api_version):
    attempt = 1
    current_query = query.strip()

    while attempt <= max_attempts:
        con.sql("SET enable_progress_bar_print = false")
        con.sql("SET progress_bar_time = 0")

        result_container = {"result": None, "error": None}

        def query_thread():
            try:
                result_container["result"] = con.execute(current_query).fetchdf()
            except duckdb.InterruptException:
                result_container["error"] = f"Query interrupted after timeout of {TIMEOUT_SECONDS} seconds."
            except Exception as e:
                result_container["error"] = str(e)

        thread = threading.Thread(target=query_thread)
        thread.start()

        start_time = time.time()
        while thread.is_alive():
            elapsed = time.time() - start_time
            if elapsed > TIMEOUT_SECONDS:
                con.interrupt()
                thread.join()
                return f"Query execution timed out after {TIMEOUT_SECONDS} seconds.", attempt, "query runs forever"
            time.sleep(0.1)

        if result_container["error"]:
            error_message = result_container["error"].lower()
            if "syntax" not in error_message and "parser" not in error_message and "binder" not in error_message:
                return f"Non-syntax error: {result_container['error']}", attempt, current_query

            print(current_query)
            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {result_container['error']}")

            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {result_container['error']}", attempt, current_query

            message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {result_container['error']}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            corrected_query = get_ai_response(message, test_model, provider, api_key, azure_endpoint, azure_api_version)

            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query : {corrected_query}", attempt, current_query

            current_query = corrected_query.strip()
            attempt += 1
        else:
            return result_container["result"], attempt, current_query

    return "Unexpected error or loop termination", attempt, current_query


In [22]:
def ask_question(questions, test_model,provider,api_key=None, azure_endpoint=None, azure_api_version=None):
    results_data = [] 
    for i, x in enumerate(questions):
        print(f"Question {i+1}: {x}") # Keep or remove print as needed
        start_time = time.time()
        sql_query_or_error = get_ai_response(x, test_model, provider, api_key, azure_endpoint, azure_api_version)
        print(sql_query_or_error) # Keep or remove print as needed
        query_result_data_json = [] # Initialize as an empty list for JSON result
        attempts_count = None
        error_details = None # Initialize error_details
        if sql_query_or_error is None or sql_query_or_error.startswith("Error"):
            # If get_ai_response returned an error or None, store the error string
            error_message = sql_query_or_error if sql_query_or_error is not None else "AI response was None"
            # print(f"Failed to get query from AI: {error_message}") # Keep or remove print
            # print("Execution: SKIPPED (AI error)") # Keep or remove print
            error_details = f"AI Error: {error_message}" # Store error details
            # query_result_data_json remains empty []
            result_row_count = 0 # Result count is 0 on error
        else:
            result_from_execution, attempts_count,query_returned = execute_sql_with_retry(sql_query_or_error, test_model,provider,api_key, azure_endpoint, azure_api_version)

            display(result_from_execution)
            is_successful = isinstance(result_from_execution, pd.DataFrame)

            if is_successful:
                print("Execution: SUCCESS") # Keep or remove print
                query_result_data_json = result_from_execution.to_dict('records')
                error_details = None # No error details on success
                result_row_count = len(result_from_execution) # Calculate row count
            else:
                print("Execution: FAILED") # Keep or remove print
                #query_result_data_json remains empty []
                error_details = f"Execution Error: {result_from_execution}" # Store error details
                result_row_count = 0 # Result count is 0 on failure


        end_time = time.time()
        duration = round(end_time - start_time, 2)
        print(f"\nExecution Time: {duration:.2f} seconds") # Keep or remove print
        print(f" ############################### ") # Keep or remove print
        results_data.append({
            "model" : test_model,
            "SF" : SF,
            "timestamp": timestamp,
            "nbr": i + 1,
            "question": x,
            "duration_s": duration, 
            "sql_query": query_returned, 
            "attempts": attempts_count,
            "result": query_result_data_json, 
            "result_count": result_row_count, 
            "error_details": error_details 
        })
    log_dir = output_dir +"/log"
    os.makedirs(log_dir, exist_ok=True)
    sanitized_model = re.sub(r'[\\/*?:"<>|]', '_', test_model)
    output_filename = f"{timestamp}_{sanitized_model}.json"
    output_path = os.path.join(log_dir, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4)
            f.flush()
            os.fsync(f.fileno())
        return f"Successfully processed {len(questions)} questions. Results saved to {output_path}"
    except IOError as e:
        return f"Error saving results to {output_path}: {e}"
    except Exception as e:
        return f"An unexpected error occurred during file saving: {e}"

In [23]:
def display_side_by_side(nbr, model1, model2):
    print(f"question {nbr} : " + duckdb.sql(f" select question from results_filtered where nbr = {nbr}  ").fetchone()[0])
    try:
        df1 = con.sql(duckdb.sql(f""" select sql_query from results_filtered where nbr = {nbr} and model = '{model1}' """).fetchone()[0]).df()        
        df2 = con.sql(duckdb.sql(f""" select sql_query from results_filtered where nbr = {nbr} and model = '{model2}' """).fetchone()[0]).df() 
        side_by_side = pd.concat([df1, df2], axis=1, keys=[model1, model2])
        display(side_by_side)
    except Exception as e:
        print(f"Error executing query for nbr {nbr}: {e}")

In [24]:
# get the questions from GitHub
try:
    response = requests.get(questions_url)
    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
    questions = json.loads(response.text)

    print("Successfully retrieved questions:")
except requests.exceptions.RequestException as e:
    print(f"Error retrieving file from GitHub: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

Successfully retrieved questions:


In [25]:
%%time
#ask_question(questions,"o3-mini","azure_foundry",os.getenv("o3_mini_key"), os.getenv("llm_endpoint"),"2025-01-01-preview" )

CPU times: total: 0 ns
Wall time: 0 ns


In [26]:
%%time
#ask_question(questions,model2,"ollama")

CPU times: total: 0 ns
Wall time: 0 ns


# Check Results

In [27]:
duckdb.sql(f""" select *,
           cardinality(result[1]) as nbr_columns,
            [result_count , nbr_columns] as result_shape,
            hash(flatten(array_transform(result, x -> json_extract(json(x), '$.*'))) )  as output
            from read_json_auto('{output_dir}/log/*.json') where model in ('{model1}','{model2}') and SF = '{SF}' """).to_view("results")
try:
 duckdb.sql(f""" install excel ; load excel ;copy results to '{output_dir}/test.xlsx' (format 'xlsx', header 'true', overwrite)  """)
except Exception as e:
    print(f"Error exporting to Excel: {e}")
    print("close the excel file or tey again.")
# check number of rows and column returned by each model, Null means SQL error
duckdb.sql(f""" 
           with zzzz as (select nbr,question,model,output , timestamp from results )
           pivot( select nbr,model,count(distinct(output)) as result_shape , count(distinct(timestamp)) as nbr_runs from zzzz group by all) 
           on model using min(result_shape) as resultsets ,min(nbr_runs) as nbr_runs order by nbr
        """).show(max_width=120)

Error exporting to Excel: IO Error: Could not move file: The process cannot access the file because it is being used by another process.

close the excel file or tey again.
┌───────┬────────────────────┬──────────────────┬──────────────────────────┬────────────────────────┐
│  nbr  │ o3-mini_resultsets │ o3-mini_nbr_runs │ qwen3:30b-a3b_resultsets │ qwen3:30b-a3b_nbr_runs │
│ int64 │       int64        │      int64       │          int64           │         int64          │
├───────┼────────────────────┼──────────────────┼──────────────────────────┼────────────────────────┤
│     1 │                  1 │                5 │                        1 │                      6 │
│     2 │                  1 │                5 │                        1 │                      6 │
│     3 │                  1 │                5 │                        1 │                      6 │
│     4 │                  1 │                5 │                        1 │                      6 │
│     5 │  

Check different results for the same Question and Models

In [28]:
Model_to_use = model2

In [29]:
df = duckdb.sql(f"""
    with xxx as (
        select nbr, output, timestamp
        from results
        where model = '{Model_to_use}'
        qualify count(distinct output) over(partition by nbr) > 1
        order by nbr
    )
    select nbr, output, min(timestamp) as timestamp
    from xxx
    group by all
    order by nbr
""").df()

# Iterate through each unique nbr
for nbr in df['nbr'].unique():
    # Print the question for the current nbr
    question = duckdb.sql(f"select question from results where nbr = {nbr}").fetchone()[0]
    print(f"Question {nbr}: {question}")
    
    # Get the timestamps and outputs for the current nbr from the query
    nbr_rows = df[df['nbr'] == nbr][['output', 'timestamp']].sort_values(by='timestamp', ascending=False)
    
    # Collect DataFrames for each timestamp
    dfs = []
    labels = []
    for _, row in nbr_rows.iterrows():
        timestamp = row['timestamp']
        try:
            # Get the SQL query for the given nbr, model, and timestamp
            sql_query = duckdb.sql(f"""
                select sql_query
                from results
                where nbr = {nbr}
                and model = '{Model_to_use}'
                and timestamp = '{timestamp}'
            """).fetchone()[0]
            
            # Execute the SQL query and convert to DataFrame
            df_temp = con.sql(sql_query).df()
            dfs.append(df_temp)
            labels.append(f"Timestamp: {timestamp}")
        except Exception as e:
            print(f"Error executing query for nbr {nbr}, timestamp {timestamp}: {e}")
    
    # Display DataFrames side by side
    if dfs:
        try:
            # Concatenate DataFrames horizontally
            combined_df = pd.concat(dfs, axis=1, keys=labels)
            display(combined_df)
        except Exception as e:
            print(f"Error combining DataFrames for nbr {nbr}: {e}")
    else:
        print(f"No valid DataFrames to display for nbr {nbr}")

Question 5: What are the different item categories available? Order alphabetically by category name.


,Timestamp: 20250513_135916,Timestamp: 20250513_114023,Timestamp: 20250512_190159
,category_name,item_category,item_category
0,Home,Men,Books
1,Electronics,Home,Children
2,Jewelry,Electronics,Electronics
3,Men,Children,Home
4,Sports,Music,Jewelry
5,Books,Women,Men
6,None,Sports,Music
7,Shoes,Books,Shoes
8,Music,None,Sports


Question 9: Compare total sales from preferred customers versus non-preferred customers, ordered by total sales


Timestamp: 20250512_193907               Timestamp: 20250512_190159  \
               customer_type   total_sales      c_preferred_cust_flag   
0              Non-Preferred  2.461229e+08                          N   
1                  Preferred  2.456942e+08                          Y   
2                    Unknown  1.625229e+07                       None   

                 
    total_sales  
0  2.461229e+08  
1  2.456942e+08  
2  1.625229e+07

Question 12: Calculate the return rate for each item category, ordered alphabetically by item category name.


...

Timestamp: 20250513_114023                                            \
          customer_age_group sale_day_type   total_sales total_returns   
0                      30-45       Weekday  8.222361e+07    4280256.91   
1                      30-45       Weekend  3.225223e+07    1553206.42   
2                    Over 45       Weekday  1.565045e+08    8147779.85   
3                    Over 45       Weekend  6.436525e+07    3224625.39   
4                   Under 30       Weekday  1.096806e+08    5516920.95   
5                   Under 30       Weekend  4.428833e+07    2325349.37   

              Timestamp: 20250512_201025                                       \
  return_rate                  age_group day_type   total_sales total_returns   
0    5.205630                      30-45  Weekday  8.222361e+07    4280256.91   
1    4.815810                      30-45  Weekend  3.225223e+07    1553206.42   
2    5.206100                    Over 45  Weekday  1.565045e+08    8147779.85   
3    5.009885                    Over 45  Weekend  6.436525e+07    3224625.39   
4    5.029986                   Under 30  Weekend  4.428833e+07    2325349.37   
5    5.250479                   Under 30  Weekday  1.096806e+08    5516920.95   

              Timestamp: 20250512_190159                                   \
  return_rate              sale_day_type customer_age_group   total_sales   
0    5.205630                    Weekday              30-45  8.222361e+07   
1    4.815810                    Weekend              30-45  3.225223e+07   
2    5.206100                    Weekday            Over 45  1.565045e+08   
3    5.009885                    Weekend            Over 45  6.436525e+07   
4    5.250479                    Weekday           Under 30  1.096806e+08   
5    5.029986                    Weekend           Under 30  4.428833e+07   

                             
  total_returns return_rate  
0    4280256.91    5.205630  
1    1553206.42    4.815810  
2    8147779.85    5.206100  
3    3224625.39    5.009885  
4    5516920.95    5.029986  
5    2325349.37    5.250479

Question 18: Which item brand has shown the largest decrease in its return rate when comparing the average rate of 2001 to 2002, specifically for stores located in the 'TN' state? (Order by the decrease in return rate descending, and then by brand name alphabetically for ties).


Timestamp: 20250512_193907                                    \
                       i_brand return_rate_2001 return_rate_2002   
0         exportiunivamalg #12        21.075982         0.000000   
1           brandunivamalg #11        15.292578         0.000000   
2         exportiamalgamalg #5        13.916474         0.843750   
3          exportiunivamalg #3        18.199075         6.369521   
4            maxiunivamalg #15        10.735027         0.000000   
..                         ...              ...              ...   
375                        NaN              NaN              NaN   
376                        NaN              NaN              NaN   
377                        NaN              NaN              NaN   
378                        NaN              NaN              NaN   
379                        NaN              NaN              NaN   

                         Timestamp: 20250512_190159                   \
    return_rate_decrease                      brand return_rate_2001   
0              21.075982       exportiunivamalg #12        21.075982   
1              15.292578         brandunivamalg #11        15.292578   
2              13.072724       exportiamalgamalg #5        13.916474   
3              11.829555        exportiunivamalg #3        18.199075   
4              10.735027          maxiunivamalg #15        10.735027   
..                   ...                        ...              ...   
375                  NaN           edu packbrand #6         2.603178   
376                  NaN      importoamalgamalg #11         0.000000   
377                  NaN           edu packbrand #7         0.000000   
378                  NaN         amalgunivamalg #10         2.136076   
379                  NaN         corpamalgamalg #16         0.000000   

                                              
    return_rate_2002 decrease_in_return_rate  
0           0.000000               21.075982  
1           0.000000               15.292578  
2           0.843750               13.072724  
3           6.369521               11.829555  
4           0.000000               10.735027  
..               ...                     ...  
375        14.637866              -12.034688  
376        12.267339              -12.267339  
377        12.906471              -12.906471  
378        15.408743              -13.272667  
379        13.579079              -13.579079  

[380 rows x 8 columns]

Question 19: For each item class, compare the average net sales value per sales transaction between preferred and non-preferred customers. Order by average net sales descending, and then by item class name alphabetically for ties.


Timestamp: 20250513_114023                              \
                    item_class customer_type avg_net_sales   
0                      country             N       -247.39   
1                       accent             N           NaN   
2                       accent             Y           NaN   
3                       accent          None           NaN   
4                  accessories          None           NaN   
..                         ...           ...           ...   
295               womens watch          None           NaN   
296               womens watch             Y           NaN   
297                       None             N           NaN   
298                       None          None           NaN   
299                       None             Y           NaN   

    Timestamp: 20250512_221139                                   \
                    item_class preferred_customer avg_net_sales   
0                      country                  N       -247.39   
1                       accent                  N           NaN   
2                       accent                  Y           NaN   
3                       accent               None           NaN   
4                  accessories               None           NaN   
..                         ...                ...           ...   
295               womens watch               None           NaN   
296               womens watch                  Y           NaN   
297                       None                  N           NaN   
298                       None               None           NaN   
299                       None                  Y           NaN   

    Timestamp: 20250512_201025                                   \
                    item_class preferred_customer avg_net_sales   
0                      country                  N       -247.39   
1                       accent                  N           NaN   
2                       accent                  Y           NaN   
3                       accent               None           NaN   
4                  accessories               None           NaN   
..                         ...                ...           ...   
295               womens watch                  N           NaN   
296               womens watch                  Y           NaN   
297                       None               None           NaN   
298                       None                  N           NaN   
299                       None                  Y           NaN   

    Timestamp: 20250512_193907                             \
                    item_class is_preferred avg_net_sales   
0                      country            N       -247.39   
1                       accent            N           NaN   
2                       accent            Y           NaN   
3                       accent         None           NaN   
4                  accessories            N           NaN   
..                         ...          ...           ...   
295               womens watch            N           NaN   
296               womens watch            Y           NaN   
297                       None         None           NaN   
298                       None            N           NaN   
299                       None            Y           NaN   

    Timestamp: 20250512_190159                     \
                    item_class preferred_customer   
0                      earings               None   
1                       custom               None   
2                     monitors               None   
3                         None               None   
4                    self-help               None   
..                         ...                ...   
295                 mattresses               None   
296                   business               None   
297                     hockey               None   
298                   wireless               None   
299                    

Compare the two Models

In [30]:
duckdb.sql(f""" 
        create or replace temp table results_filtered as
           from results  where model = '{model1}' and timestamp = (select max(timestamp) from results where model = '{model1}') 
           union all 
           from results  where model = '{model2}' and timestamp = (select max(timestamp) from results where model = '{model2}')
""")
# execusion time in seconds
duckdb.sql(f""" 
           pivot( select nbr as question ,model,result_shape , attempts from results_filtered ) 
           on model using min(result_shape) as result,  min(attempts) as attempts order by question
        """).show(max_width=130)

┌──────────┬────────────────┬──────────────────┬──────────────────────┬────────────────────────┐
│ question │ o3-mini_result │ o3-mini_attempts │ qwen3:30b-a3b_result │ qwen3:30b-a3b_attempts │
│  int64   │    int128[]    │      int64       │       int128[]       │         int64          │
├──────────┼────────────────┼──────────────────┼──────────────────────┼────────────────────────┤
│        1 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        2 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        3 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        4 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        5 │ [11, 1]        │                1 │ [11, 1]              │                      1 │
│        6 │ [6, 2]         │                1 │ [6, 2]               │                      1 │
│        7 │ [1, 2]         │ 

In [31]:
different_results = duckdb.sql(f"""
    SELECT 
        distinct(nbr) 
    FROM results_filtered 
    WHERE nbr in
    (
    select nbr from results_filtered 
    group by all
    having count(distinct(output)) >  1 
    )
    order by nbr
""").df()['nbr'].tolist()
print(f"Different results for {len(different_results)} questions: {different_results}")
good_results = list(set(list(range(1, 21))) - set(different_results))
print(f"Exact resultsets for {len(good_results)} questions: {good_results}")


Different results for 8 questions: [5, 12, 14, 15, 16, 17, 18, 19]
Exact resultsets for 12 questions: [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 20]


In [32]:
for nbr in list(range(1, 21)):
    display_side_by_side(nbr, model1, model2)

question 1 : What is the overall total sales revenue?


,o3-mini,qwen3:30b-a3b
,total_sales,total_sales
0,5.143009e+08,5.143009e+08


question 2 : What is the total number of items sold across all transactions?


,o3-mini,qwen3:30b-a3b
,total_quantity,total_quantity
0,13939520.0,13939520.0


question 3 : What is the total monetary value of all returned items?


...

o3-mini                                    \
                  i_brand return_rate_2001 return_rate_2002   
0    exportiamalgamalg #5        13.916474         0.843750   
1     exportiunivamalg #3        18.199075         6.369521   
2       maxiunivamalg #13         9.189426         1.439496   
3        corpunivamalg #7         8.067342         0.487251   
4          scholarmaxi #8         7.526645         0.476072   
..                    ...              ...              ...   
375                   NaN              NaN              NaN   
376                   NaN              NaN              NaN   
377                   NaN              NaN              NaN   
378                   NaN              NaN              NaN   
379                   NaN              NaN              NaN   

                                     qwen3:30b-a3b                   \
    decrease_in_return_rate                i_brand return_rate_2001   
0                 13.072724   exportiunivamalg #12        21.075982   
1                 11.829555     brandunivamalg #11        15.292578   
2                  7.749930   exportiamalgamalg #5        13.916474   
3                  7.580091    exportiunivamalg #3        18.199075   
4                  7.050572      maxiunivamalg #15        10.735027   
..                      ...                    ...              ...   
375                     NaN       edu packbrand #6         2.603178   
376                     NaN  importoamalgamalg #11         0.000000   
377                     NaN       edu packbrand #7         0.000000   
378                     NaN     amalgunivamalg #10         2.136076   
379                     NaN     corpamalgamalg #16         0.000000   

                                              
    return_rate_2002 decrease_in_return_rate  
0           0.000000               21.075982  
1           0.000000               15.292578  
2           0.843750               13.072724  
3           6.369521               11.829555  
4           0.000000               10.735027  
..               ...                     ...  
375        14.637866              -12.034688  
376        12.267339              -12.267339  
377        12.906471              -12.906471  
378        15.408743              -13.272667  
379        13.579079              -13.579079  

[380 rows x 8 columns]

question 19 : For each item class, compare the average net sales value per sales transaction between preferred and non-preferred customers. Order by average net sales descending, and then by item class name alphabetically for ties.


o3-mini                                     qwen3:30b-a3b  \
            i_class c_preferred_cust_flag avg_net_sales    item_class   
0           earings                  None   2532.885873       earings   
1            custom                  None   2363.180714        custom   
2          monitors                  None   2272.177463      monitors   
3         self-help                  None   2269.849091          None   
4    entertainments                  None   2174.550328     self-help   
..              ...                   ...           ...           ...   
295        business                  None   1378.109839    mattresses   
296        wireless                  None   1293.797018      business   
297     consignment                  None   1272.868095        hockey   
298          hockey                  None   1225.556438      wireless   
299            arts                  None   1143.317292          arts   

                                                      
    preferred_customer avg_net_sales_per_transaction  
0                 None                   2704.606949  
1                 None                   2406.147636  
2                 None                   2272.177463  
3                 None                   2270.826970  
4                 None                   2269.849091  
..                 ...                           ...  
295               None                   1457.595789  
296               None                   1400.701803  
297               None                   1355.539697  
298               None                   1340.844182  
299               None                   1167.643191  

[300 rows x 6 columns]

question 20 : show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending.


o3-mini                                       qwen3:30b-a3b  \
  s_store_name    net_sales avg_net_sales pct_vs_avg    store_name   
0        ought  481787676.8   481787676.8      100.0         ought   

                                                          
     net_sales avg_net_sales percentage_of_avg_net_sales  
0  481787676.8   481787676.8                       100.0